<a href="https://colab.research.google.com/github/DCabrini/sistema_de_assist-ncia_virtual/blob/main/sistema_de_assist%C3%AAncia_virtual.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Verificando se a GPU foi inicializada
import tensorflow as tf
devive_name = tf.test.gpu_device_name()
print(devive_name)

# Retorno esperado "/device:GPU:0"

In [2]:
!pip install gTTS
!pip install SpeechRecognition
!pip install wikipedia
!pip install geopy


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.9/32.9 MB 24.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11679 sha256=0599de8090a6394ad9ae4f0d56e6875979ee4e6c8d58bae51b78adf3f2f492a3
  Stored in directory: /root/.cache/pip/wheels/8f/ab/cb/45ccc40522d3a1c41e1d2ad53b8f33a62f394011ec38cd71c6
Successfully built wikipedia


In [3]:
from gtts import gTTS
import os

def texto_para_fala(texto):
    tts = gTTS(text=texto, lang='pt')
    tts.save("fala.mp3")
    os.system("mpg321 fala.mp3")  # Reproduz o áudio


In [4]:
import speech_recognition as sr

def fala_para_texto():
    r = sr.Recognizer()
    with sr.Microphone() as source:
        print("Aguardando comando...")
        r.adjust_for_ambient_noise(source)
        audio = r.listen(source)
        try:
            comando = r.recognize_google(audio, language='pt-BR')
            print("Você disse: " + comando)
            return comando
        except sr.UnknownValueError:
            print("Não entendi, pode repetir?")
        except sr.RequestError:
            print("Erro ao conectar com o serviço de reconhecimento de fala.")


In [5]:
import wikipedia

def buscar_wikipedia(termo):
    try:
        resultado = wikipedia.summary(termo, sentences=1)
        return resultado
    except wikipedia.exceptions.DisambiguationError as e:
        return "O comando está ambíguo, por favor, refine sua busca."
    except wikipedia.exceptions.HTTPTimeoutError:
        return "Erro de conexão com o Wikipedia."


In [6]:
from geopy.geocoders import Nominatim

def localizar_farmacia(endereco_atual):
    geolocator = Nominatim(user_agent="assistente_virtual")
    location = geolocator.geocode(endereco_atual)
    print(f"Latitude e Longitude de {endereco_atual}: {location.latitude}, {location.longitude}")
    # Aqui você poderia integrar com uma API de mapas para buscar farmácias


In [7]:
def acionar_comando():
    comando = fala_para_texto()
    if comando:
        if "wikipedia" in comando:
            termo = comando.replace("pesquisar no wikipedia", "").strip()
            resultado = buscar_wikipedia(termo)
            texto_para_fala(resultado)
        elif "youtube" in comando:
            import webbrowser
            webbrowser.open("https://www.youtube.com")
            texto_para_fala("Abrindo o YouTube.")
        elif "farmácia" in comando:
            endereco = "sua localização atual"  # Aqui você pode adicionar o código de geolocalização
            localizar_farmacia(endereco)
            texto_para_fala("Localizando farmácia mais próxima.")
        else:
            texto_para_fala("Comando não reconhecido. Tente novamente.")
